<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch15-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd

# 긴 시퀀스 다루기
- 문제점 및 해결 방법(ch15-1에서 왜 tanh가 디폴트인지 알아보면서 살짝 다룸)
  - gradient 소실 또는 폭주 문제 -> 불안정한 학습
  - RNN이 긴 시퀀스를 처리할 때 입력의 첫 부분을 조금씩 잊어버리는 문제 
  - 해결 방법
    - [gradient clipping](https://sanghyu.tistory.com/87)
    - 낮은 learning rate
    - 정규화 
    - 장기 메모리 셀 ex) LSTM, GRU(LSTM변종) 등
<br><br>
- RNN에 잘맞는 종류의 정규화: <strong>층 정규화</strong>(ch13-2에서 살짝 다룸)
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcdaNCJ%2FbtqAa7RVwhD%2FP60H1iai2Mxgvu09TGQpv0%2Fimg.png" width=80% height=80%/>

  - 즉, 배치 정규화는 배치 차원에 대해 정규화, 층 정규화는 <strong>특성 차원에 대해 정규화</strong>
  - 그래서 훈련과 테스트에서 동일한 방식으로 작동한다.
    - 배치 정규화는 훈련 세트의 모든 세트에 대한 특성 통계를 추정하기 위해 지수 이동 평균이 필요했음

## 메모리 셀 안에 층 정규화 구현

In [ ]:
class LNSimpleRNNCell(keras.layers.Layer):
  def __init__(self, units, activation="tanh", **kwargs):
    super().__init__(**kwargs)
    self.state_size=units
    self.output_size=units
    self.simple_rnn_cell=keras.layers.SimpleRNNCell(units,activation=None)#===포인트: activation=None이다(활성화 함수에 넣기 전에 정규화하려고)
    self.layer_norm=keras.layers.LayerNormalization()
    self.activation=keras.activations.get(activation)
  
  def call(self, inputs, states):#====현재 타임 스텝의 inputs와 이전 타임 스텝의 hidden states(h_t-1)
    outputs, new_states= self.simple_rnn_cell(inputs, states)
    norm_outputs=self.activation(self.layer_norm(outputs)) #정규화하고 난 후의 값을 활성화 함수에 대입
    return norm_outputs, [norm_outputs] #===두 개인 이유: 하나는 출력, 하나는 새로운 은닉 상태(h_t)가 된다


- 사용자 정의 셀 적용

In [ ]:
model=keras.models.Sequential([
                               keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
                               keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
                               keras.layers.TimeDistributed(keras.layers.Dense(10))
])

-----------
위 처럼 keras.layers.RNN층을 만들어서 LNSimpleRNNCell의 객체를 전달하면 된다.
- 만약 타임 스텝 사이에 dropout을 적용하고 싶다면?
  - 위처럼 드롭아웃을 적용하는 사용자 정의 셀을 만들어도 되긴하는데, 
  - keras.layers.SimpleRNN(recurrent_dropout=0.5) 매개변수를 지원함

## LSTM(Long Short-Term Memory) 셀
- RNN을 거치면서 데이터가 변환되므로 일부 정보는 매 훈련 스텝 후 사라짐
- 그래서 어느 정도 시간이 지나면 사실상 <strong>첫 번째 입력의 흔적이 사라짐</strong>
- 몇몇문제는 위 특징이 심각한 문제가 될 수 있음 그래서 <strong>장기 메모리</strong>를 가진 여러 종류의 셀이 연구됨
- 장기 메모리를 가진 셀에서 가장 인기있는게 <strong>LSTM셀</strong>

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile5.uf.tistory.com%2Fimage%2F9905CF385BD5F5EC027F20" />

- 핵심 아이디어: 네트워크가 <strong>장기 상태에 저장</strong>할 것, <strong>버릴 것</strong>, 그리고 <strong>읽어들일 것</strong>을 학습하는 것
- h_t: 단기 상태(short-term state)
- c_t: 장기 상태(long-term state) 
- 로직
  - 장기기억 c_t-1은 왼->오른쪽으로 관통하면서 <strong>삭제 게이트(f_t)를 지나 일부 기억을 잃는다.</strong>
  - 그런 다음 <strong>입력 게이트(i_t)에서 새로운 기억 일부가 추가</strong>된다.
  - 만들어진 c_t는 다른 <strong>추가 변환 없이 바로 출력</strong>으로 보내진다.
  - h_t는 c_t가 복사되어 <strong>tanh함수</strong>를 거친 후 <strong>결과 게이트(o_t)</strong>에서 걸러서 만들어진다.
    - o_t는 장기 상태의 어느 부분을 읽어서 이 타임 스텝의 h_t와 y_t로 출력해야 하는지 제어한다.
<br><br><br>

### 적용 방법
- SImpleRNN 대신 LSTM 층 사용
  - 이 방법을 일반적으로 많이 사용(GPU에서 실행할 때 최적화된 구현을 사용해서)

In [ ]:
model2= keras.models.Sequential([
                                 keras.layers.LSTM(20,return_sequences=True, input_shape=[None, 1]),
                                 keras.layers.LSTM(20, return_sequences=True),
                                 keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 20)          1760      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 20)          3280      
_________________________________________________________________
time_distributed (TimeDistri (None, None, 10)          210       
Total params: 5,250
Trainable params: 5,250
Non-trainable params: 0
_________________________________________________________________


- RNN층에 LSTMCell을 매개변수로 지정할 수도 있음
  - 이 방법은 사용자 정의 셀을 정의할 때 많이 사용

In [ ]:
model3 = keras.models.Sequential([
                                  keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True, input_shape=[None, 1]),
                                  keras.layers.RNN(keras.layers.LSTMCell(20), return_sequences=True),
                                  keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, None, 20)          1760      
_________________________________________________________________
rnn_2 (RNN)                  (None, None, 20)          3280      
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 10)          210       
Total params: 5,250
Trainable params: 5,250
Non-trainable params: 0
_________________________________________________________________


### 핍홀 연결
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F99020E395BD5F62C049EFE"/>

- LSTM의 변종으로, <strong>LSTM에 추가적인 연결</strong>이 있다(핑크 선)
  - 이전 장기 기억 상태인 <strong>c_t-1이 삭제 게이트 f_t, 입력 게이트 i_t에</strong> 입력으로 추가된다.
  - 현재 장기 기억 상태인 <strong>c_t는 출력 게이트 o_t에</strong> 입력으로 추가된다.
- 게이트 제어기에 <strong>장기 상태를 조금 노출시켜 좀 더 많은 문맥을 감지</strong>하게 만들 수 있다는 아이디어에서 제안한 것
  - 하지만 늘 성능이 향상되진 않으니 적용해서 확인해봐야함

- 적용
 - keras.experimental.PeepholeLSTMCell 지원

In [ ]:
keras.layers.RNN(keras.experimental.PeepholeLSTMCell(20)) #=== keras.layers.LSTMCell은 핍홀을 지원하지 않음

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/recurrent.py:2560: UserWarning: `tf.keras.experimental.PeepholeLSTMCell` is deprecated and will be removed in a future version. Please use tensorflow_addons.rnn.PeepholeLSTMCell instead.
  warnings.warn('`tf.keras.experimental.PeepholeLSTMCell` is deprecated '


### GRU(Gated Recurrent Unit)셀

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile7.uf.tistory.com%2Fimage%2F99F0EC3E5BD5F6460255CF"/>

- LSTM의 변종으로, LSTM의 간소화 버전이다.
  - 기존의 <strong>두 상태 벡터(c_t, h_t)가 하나로</strong> 합쳐졌다. -> h_t
  - 하나의 게이트 제어기 <strong>z_t가 삭제 게이트와 입력 게이트를 모두 제어</strong>한다.
    - 게이트 제어기가 <strong>1</strong>을 출력하면 삭제 게이트가 열리고 입력 게이트가 닫힘
    - <strong>0</strong>을 출력하면 입력 게이트가 열리고 삭제 게이트가 닫힘
  - <strong>출력 게이트가 없어짐</strong>
    - 대신 이전 상태의 <strong>어느 부분이 주 층(g_t)에 노출될지 제어</strong>하는 새로운 게이트 제어기 r_t가 있음

- 적용
  - keras.layers.GRUCell셀을 기반으로 한 keras.layers.GRU층 제공

In [ ]:
keras.layers.GRU(20)

## 1D 합성곱 층을 사용해 시퀀스 처리하기
- LSTM과 GRU셀은 RNN 성공의 주역 중 하나로, RNN보다 훨씬 긴 시퀀스를 다룰 수 있다.  
- 하지만 매우 제한적인 단기 기억을 가진다.
  - <strong>100 타임 스텝 이상의 시퀀스에서 장기 패턴</strong>을 학습하는데 어려움이 있음
  -  <strong>해결 방법1: 1D conv를 사용해서 입력 시퀀스를 짧게 줄이는 것 </strong>
  - <strong> 해결 방법2: 팽창 비율을 가진 CNN구조 (WaveNet)</strong>

- 적용

In [ ]:
model4=keras.models.Sequential([
                                keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid", input_shape=[None,1]), #====strides=2로 해서 입력 시퀀스를 두 배로 다운샘플링
                                keras.layers.GRU(20, return_sequences=True),
                                keras.layers.GRU(20, return_sequences=True),
                                keras.layers.TimeDistributed(keras.layers.Dense(10))

])

In [ ]:
model4.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, None, 20)          100       
_________________________________________________________________
gru_1 (GRU)                  (None, None, 20)          2520      
_________________________________________________________________
gru_2 (GRU)                  (None, None, 20)          2520      
_________________________________________________________________
time_distributed (TimeDistri (None, None, 10)          210       
Total params: 5,350
Trainable params: 5,350
Non-trainable params: 0
_________________________________________________________________


### WaveNet
<img src="https://paperswithcode.com/media/methods/Screen_Shot_2020-05-24_at_12.11.35_AM_uO4kv1I.png" width=50% height=60%/>

- <strong>층마다 팽창 비율(각 뉴런의 입력이 떨어져 있는 간격)을 두 배로</strong> 늘리는 1D합성곱 층을 쌓는다.
- 즉, 첫 번째 층이 한 번에 1개의 타임 스텝만 보고, 그 다음 층이 2개 , 그 다음 층이 4개의 타임 스텝을 보고, 그 다음 층이 8개 ... 
  - 이런식으로 하면 <strong>하위 층은 단기 패턴</strong>을 학습하고
  - <strong>상위 층은 장기 패턴</strong>을 학습하게 된다.


- 적용

In [ ]:
model5= keras.models.Sequential()
model5.add(keras.layers.InputLayer([None, 1]))
for rate in (1,2,4,8)*2:
  model5.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal", activation='relu', dilation_rate=rate))

model5.add(keras.layers.Conv1D(filters=10, kernel_size=1))

In [ ]:
model5.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, None, 20)          60        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 20)          820       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 20)          820       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 20)          820       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 20)          820       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 20)          820       
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 20)         

#### causal padding?

<img src="https://images.velog.io/images/changdaeoh/post/e81e4268-ebd3-430f-a8c9-7314063e90bc/image.png"/>

- kernel size - 1개 만큼 <strong>왼쪽으로만 0</strong>을 패딩으로 추가하여 <strong>필터가 현재와 과거의 값들에만 적용</strong>되도록 구현
  - 더 정확히는 dilation_rate  * (kernel_size-1)만큼 왼쪽에 0을 패딩으로 추가

- 층에 추가한 이 패딩 덕분에 입력 시퀀스의 길이와 동일한 시퀀스를 출력한다.
  - 즉, 전체 시퀀스를 타깃으로 사용할 수 있다.

참고)
[changdaeoh님 블로그](https://velog.io/@changdaeoh/Convolutionforsequence)

----------
- 오디오에는 1초에 수만 개의 타임 스텝이 포함될 수 있는데, 여러 오디오 문제에서 최상의 성능을 달성한 모델이다. (그래서 이름이 WaveNet)
  - 참고로 LSTM이나 GRU는 이렇게 긴 시퀀스를 다룰 수 없음

# 연습문제
### 시퀀스-투-시퀀스 / 시퀀스-투-벡터 / 벡터-투-시퀀스 RNN을 사용한 애플리케이션에는 어떤 것들이 있나요?
- 시퀀스-투-시퀀스
  - 날씨 예측(또는 다른 시계열 관련 문제)
  - 기계 번역(인코더-디코더 구조 사용)
  - 비디오 캡션 생성
  - 스피치 투 텍스트
  - 음악 생성(또는 다른 시퀀스 생성)
  - 노래의 화음 식별
- 시퀀스-투-벡터
  - 음악 샘플을 장르로 구분하기
  - 책 후기에 대한 감성 분석
  - 뇌에 심은 인공칩에서 읽은 데이터를 기반으로 실어증 환자가 생각하는 단어 예측하기
  - 사용자의 영화 시청 이력을 바탕으로 보고 싶어 할 영화의 확률 예측하기
    - 이는 추천 시스템을 위해 구현 가능한 협업 필터링 애플리케이션 중 하나
- 벡터-투-시퀀스
  - 이미지 캡션 생성
  - 현재 아티스트를 기반으로 음악 플레이리스트 생성
  - 일련의 파라미터를 기반으로 한 멜로디 생성
  - 사진(자율주행 자동차의 카메라에서 찍은 동영상 프레임) 속 보행자 위치 찾기

<br><br>

### RNN 층의 입력은 얼마나 많은 차원을 가지나요? 각 차원이 표현하는 것은 무엇인가요? 출력은 어떤가요?
- RNN층의 입력 차원은 3개
  - 첫 번째 차원은 배치 크기
  - 두 번째 차원은 타임 스텝
  - 세 번째 차원은 입력 특성의 개수
  - ex) 2개의 값을 가진 타임 스텝 10개로 이루어진 시계열 5개를 담은 배치 처리 -> [5,10,2]
- 출력 차원은 RNN의 뉴런 개수와 동일(마지막 차원만 다름)
  - ex) 위 경우에서 RNN의 unit=2라면, [5,10,2]이 되겠다.


  <br><br>

### 심층 시퀀스-투-시퀀스 RNN을 만든다면 어떤 RNN층을 return_sequences=True로 설정해야 하나요? 시퀀스-투-벡터 RNN은 어떤가요?
- 케라스로 심층 시퀀스-투-시퀀스 RNN을 만드려면
  - 모든 RNN층에 return_sequences=True
- 시퀀스-투-벡터 RNN을 만드려면
  - 마지막(상위) 층만 return_sequences=False

<br><br>

### 일자별 단변량 시계열 데이터를 가지고 다음 7일을 예측하려고 합니다. 어떤 RNN구조를 사용해야 하나요?
- 가장 간단한 구조는 RNN층을 쌓고, 마지막 층만 뉴런 7개 사용
  - output을 다음 input으로 사용
- 모든 RNN층에 return_sequences=True설정
  - output을 다음 input으로 사용
  - 각 타깃 시퀀스는 타임 스텝마다 7개의 값을 가짐

  <br><br>

### RNN을 훈련할 때 주요 문제는 무엇인가요? 어떻게 이를 처리할 수 있나요?
- 불안정한 gradient와 제한적인 단기 기억 문제로, 긴 시퀀스를 다룰 때 더욱 심각
  - 해결 방법
    - 작은 learning rate
    - tanh같이 수렴하는 활성화 함수 사용
    - gradient clipping
    - 층 정규화
    - 타임 스텝마다 드롭아웃
    - LSTM
    - GRU

<br><br>

### 왜 RNN안에 1D 합성곱 층을 사용해야 하나요?
- RNN에 하나 이상의 1D합성곱 층을 사용해서 <strong>다운 샘플링하여 장기 패턴을 감지</strong>하는 데 도움이 된다.
  - WaveNet처럼 1D합성곱 층만 사용할 수 있다.
  - RNN층은 근본적으로 순차적이다. 따라서 타임 스텝 t에서의 출력을 계산하려면 이전 타임 스텝의 출력을 계산해야한다.
    - 이런 방식은 병렬화하지 못함
  - 1d 합성곱 층은 타임 스텝 간의 상태를 유지하지 않음
      - 이런 방식은 병렬화 하기 쉽다.
      - 즉, 메모리가 없다.
      - 또한 순환 층이 아니라서 불안정한 gradient(위 문제 중 하나)의 영향을 덜 받음

<br><br>

### 영상을 분류하기 위해 어떤 신경망 구조를 사용할 수 있나요?
- ex)화면 내용을 기반으로 동영상을 분류하기
  1. 초당 한 프레임을 받아 각 프레임을 합성곱 신경망에 통과 시키기
  2. 출력 시퀀스를 시퀀스-투-벡터 RNN에 주입하기
  3. 마지막에 소프트맥스 층을 통과시켜 모든 클래스에 대한 확률 구하기

- 만약 오디오도 포함시킨다면,
  1. 스트라이드 1d 합성곱 층을 쌓아 초당 수천 개의 오디오 프레임을 초당 하나로 시간 방향 해상도 줄이기(해당 출력을 초당 이미지 개수에 맞추기)
  2. 위 시퀀스-투-벡터 RNN의 입력에 연결하기